## What is this notebook?

This book makes a lead time dependent climatology to use for any forecast system. 

Why Provide a Lead-Time Dependent Model Climatology in S2S Forecasting?

In Subseasonal-to-Seasonal (S2S) forecasting, incorporating a lead-time dependent model climatology offers several advantages that contribute to improved forecast accuracy and reliability. Here’s why it’s considered a beneficial practice:

- Reduced Biases: A standard climatology applied uniformly across all lead times might not account for the dynamic nature of atmospheric conditions, especially as a model slips into its own biased climatology, which is distinct from the real-world climatology. A lead-time dependent model climatology adapts to the evolving climate, mitigating biases that arise from using fixed observed climatological values.
- Capturing Evolution: The climate system’s behavior changes as forecasts extend further into the future. A lead-time dependent climatology better captures this evolving nature, ensuring that the forecast is aligned with the expected conditions at various lead times.
- Improved Skill: Incorporating a climatology that considers the lead time enhances the forecast’s skill by accounting for the removal of the biased evolution of the model’s climate. This leads to better capturing the evolving patterns, resulting in more accurate predictions.
- Contextual Insights: A lead-time dependent climatology provides contextual insights into how the model’s climate system evolves over time. This understanding enhances the interpretation of forecast anomalies and aids in identifying significant departures from the norm.


In summary, integrating a lead-time dependent model climatology acknowledges the changing nature of the model’s climate system and how it transitions into its own attractor space. By harnessing its variability, this approach enhances the accuracy and reliability of S2S forecasts. By adapting to evolving atmospheric conditions, this approach helps create more skillful and informative predictions.

This preproccessing notebook allows a user to take their forecast and make a lead time dependent climatology in order to improve the skill of their forecasts. I strongly recommend the use of a dask cluster... otherwise this will take some time! 

Please adjust the "settings" markdown box below in order to use this notebook. 

The resulting NC file will be a lead time dependent climatology at every given day

# What you will need to run this notebook:

- 1) a directory with all of your forecast lead files in it... [/glade/scratch/wchapman/MJO_S2S_CESM2/p1/]
- - Each File must be named by forecast lead time (i.e., S2Shindcast_cesm2cam6vs_MJOvars_10sep2018.nc)
- 2) a certain amount of "coverage" of the files (i.e., you know how many files you need to make a CLIMO) it can't only be 5 

## Import packages

In [15]:
import sys
import os
import warnings
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#import collections
import pandas as pd
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# import netCDF4
# from netCDF4 import *

import cartopy.crs as ccrs
import cartopy as cart
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from scipy import interpolate
from scipy.interpolate import griddata
import time
import glob
import dask

from scipy.fftpack import fft
from scipy.fftpack import ifft
import copy
import eofs.standard as Eof_st
from eofs.multivariate.standard import MultivariateEof

from matplotlib.colors import ListedColormap
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import cmocean

from scipy.stats import gaussian_kde
from pandas import Series
from datetime import datetime, timedelta


## Settings

In [32]:
#output directory
outdir = '/glade/scratch/wchapman/MJO_S2S_CESM2/climo/'
#output file pattern:
Output_Fstring = 'MJOvars_climo_*.nc' #make sure you include a wild card (i.e. 'MJOvars_climo_*.nc')

#directory where your forecast files are stored: 
dir_s2sfiles = '/glade/scratch/wchapman/MJO_S2S_CESM2/p1/'
Fstring_s2sfiles = 'S2Shindcast_cesm2cam6vs_MJOvars_*.nc'

#number of lead time days:
num_leadays = 46

#DAYS OF YEAR FOR THE CLIMO this is the starting day np.arange(1,367) does every day. ... 
# this can be a np.array with just the desired start date of year np.array([45,66,76])
days_of_Y = np.arange(1, 367)

string_format_forecast_File="%d%b" 
# look up the strftime formats here: https://strftime.org/
## if your files are of the form XXX_10sep2018.nc you will want %d%b to define the date as 10sep
### the idea is to define this variable so that it recovers the day and month! 
####according to how you formated the ouput files. 

## Shut Down DASK Client If Necessarry

In [19]:
if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')

...shutdown client...


## Import Client for Dask Distributed Job

In [4]:
from distributed import Client
from dask_jobqueue import PBSCluster

cluster = PBSCluster(project='NAML0001',walltime='12:00:00')
cluster.scale(40)
client = Client(cluster)
client

In [49]:
def get_dates_15_days_around(day_of_year, dminus, dplus):
    # Get the current year
    current_year = datetime.now().year
    
    # Create a date object for the given day of the year
    date_obj = datetime.strptime(f"{day_of_year:03d}", "%j")
    
    # Generate the 15 days earlier and 15 days later dates
    dates_list = []
    print(date_obj)
    for delta in range(dminus, dplus):
        date = date_obj + timedelta(days=delta)
        date_str = date.strftime(string_format_forecast_File).lower()
        dates_list.append(date_str)

    return dates_list,date_obj

dates_list,date_obj = get_dates_15_days_around(10,-10,10)

1900-01-10 00:00:00


## These are Useful Functions

In [20]:
# Function to get dates 15 days before and after a given day of the year
def get_dates_15_days_around(day_of_year, dminus, dplus):
    # Get the current year
    current_year = datetime.now().year
    
    # Create a date object for the given day of the year
    date_obj = datetime.strptime(f"{day_of_year:03d}", "%j")
    
    # Generate the 15 days earlier and 15 days later dates
    dates_list = []
    for delta in range(dminus, dplus):
        date = date_obj + timedelta(days=delta)
        date_str = date.strftime(string_format_forecast_File).lower()
        dates_list.append(date_str)

    return dates_list

# Function to preprocess the dataset
def preprocess(ds):
    # Set the 'time' coordinate to a new range from 0 to 45
    ds['time'] = np.arange(0, 46)
    return ds

# Function to flatten a list of lists
def flatten_list(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

# Function to convert day of the year and year to a date string
def dayofyear_to_date_string(year, dayofyear):
    date = pd.to_datetime(f"{year}-{dayofyear}", format="%Y-%j")
    date_string = date.strftime('%Y-%m-%d')
    return date_string

def check_file_exists(filename):
    return os.path.exists(filename)


def adjust_number_forward(number):
    if number > 366:
        difference = number - 366
        return difference
    else:
        return number
    
    
def convert_dates_to_string(input_string):
    # Define the regex pattern to match various date formats
    date_pattern = r'(\d{1,2}[A-Za-z]{3}\d{4}|\d{1,2}-[A-Za-z]{3}-\d{4}|[A-Za-z]{3}[-_\s]\d{1,2}[-_\s]\d{4}|[0-3]?\d[0-1]?\d\d{2}|[0-1]?\d[0-3]?\d\d{2})'
    
    # Find all matches of the date pattern in the input string
    try:
        matches = re.findall(date_pattern, input_string)
    except:
        raise RuntimeError("the forecast files do not have a proper datestring as defined in the settings.yaml please change their names")
    
    
    # Loop through each matched date and convert it to the desired format
    for match in matches:
        # Parse the date string to a datetime object
        try:
            date_obj = datetime.strptime(match, '%b-%d-%Y')
        except ValueError:
            try:
                date_obj = datetime.strptime(match, '%d-%b-%Y')
            except ValueError:
                try:
                    date_obj = datetime.strptime(match, '%b_%d_%Y')
                except ValueError:
                    try:
                        date_obj = datetime.strptime(match, '%b-%d-%Y')
                    except ValueError:
                        try:
                            date_obj = datetime.strptime(match, '%m%d%y')
                        except ValueError:
                            try:
                                date_obj = datetime.strptime(match, '%d%b%Y')
                            except ValueError:
                                continue  # Skip if the date format doesn't match any of the known formats
        # Convert the datetime object to the desired format
        formatted_date = date_obj.strftime('%d%b%Y')
        
        # Replace the matched date in the input string with the formatted date
        input_string = input_string.replace(match, formatted_date)
    
    return input_string,matches

## Run Part 1 

This creates the lead time dependent files at each lead for each day

In [4]:
#assertion: 
assert '*' in Output_Fstring, 'Output_Fstring must contain a wildcard i.e.: MJOvars_climo_*.nc'
assert '*' in Fstring_s2sfiles, 'Fstring_s2sfiles must contain a wildcard i.e.: S2Shindcast_cesm2cam6vs_MJOvars_*.nc'

print('the files will be placed here: ', dirout+Output_Fstring.split('*')[0]+'*'+Output_Fstring.split('*')[1])

# Loop through days of the year (1 to 365) to process the data
for ii in range(1, 367):
    print(f"day of year {ii} of 366")
    #### outfile 
    year_start = 1979
    dayofyear_start = ii
    date_string = dayofyear_to_date_string(year_start, dayofyear_start)
    dirout = outdir
    outfile = dirout+Output_Fstring.split('*')[0]+date_string+Output_Fstring.split('*')[1]
    ####
    
    if check_file_exists(outfile):
        print('file exists already')
        continue 
    
    # Record the start time for elapsed time calculation
    start_time = time.time()
    
    # Get the 15 days before and after the current day of the year
    result_dates = get_dates_15_days_around(ii, -15, 16)
    
    # Find files for the specified dates and concatenate them
    FNs_dats = []
    for rsrs in result_dates:
        FNs_dats.append(sorted(glob.glob(dir_s2sfiles+Fstring_s2sfiles.split('*')[0]+rsrs+'*'+Fstring_s2sfiles.split('*')[1])))
    FNs_dats = flatten_list(FNs_dats)
    print(len(FNs_dats))
    
    # Open and preprocess the dataset using xarray
    try:
        DSmerge = xr.open_mfdataset(FNs_dats, parallel=True, combine='nested', concat_dim='bingo', preprocess=preprocess)
    except:    
        DSmerge = xr.open_mfdataset(FNs_dats, parallel=True, combine='nested', concat_dim='bingo', preprocess=preprocess)
    
    # Calculate the ensemble and temporal mean
    DSmerge = DSmerge.mean(['bingo', 'ensemble']).squeeze()
    
    # Create a new coordinate 'dayofyear' and expand the dataset with it
    data_newcoord = DSmerge.assign_coords(dayofyear='coord_value')
    data_expanded = data_newcoord.expand_dims('dayofyear')
    data_expanded['dayofyear'] = np.atleast_1d(ii)
    
    # Load the dataset into memory
    data_expanded.load()
    
    # Update the 'time' coordinate with the correct dates
    year_start = 1979
    dayofyear_start = ii
    date_string = dayofyear_to_date_string(year_start, dayofyear_start)
    data_expanded['time'] = pd.date_range(start=date_string, periods=46, freq="D")
    data_expanded2 = copy.deepcopy(data_expanded)
    # Record the end time and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"Elapsed time: {elapsed_time:.6f} seconds after the load")
    
    # Save or perform further processing
    FNs_dats
    data_expanded.attrs["forecasts_averaged_over"] = FNs_dats
    data_expanded.attrs["title"] = "MJO-related Climate Variables Climatology"
    data_expanded.attrs["source"] = "CESM2/CAM6 Model"
    data_expanded.attrs["institution"] = "National Center for Atmospheric Research (NCAR)"
    data_expanded.attrs["references"] = "Will Chapman. (2023), Journal of Climate, Vol. XX, pp. XX-XX."
    data_expanded.attrs["description"] = "This dataset contains MJO-related climate variables."
    data_expanded.attrs["author"] = "Will Chapman"
    data_expanded.attrs["creation_date"] = "2023-07-28"
    data_expanded.attrs["contact"] = "Will Chapman (wchapman@ucar.edu)"
    data_expanded.to_netcdf(outfile)
    


day of year 1 of 366
110


## Compile based on lead time:

This block compiles all of the files

In [ ]:
%%time

fnfn = sorted(glob.glob(outdir + Output_Fstring ))
DDS = xr.open_dataset(fnfn[10])
outdir = outdir
for ldld in np.arange(0,num_leadays):
    outfile = outdir+'MJOvars_lead'+f"{ldld:0{3}d}"+'_climo.nc'
    if check_file_exists(outfile):
        print('file exists already')
        continue 
        
    print(ldld)
    start_time = time.time()
    for ee,FN in enumerate(fnfn):
        DDS=xr.open_dataset(FN)
        #open the files and adjust the dates forward in time:
        if ee == 0:
            DDSmerge = DDS.isel(time=ldld).drop('time')
            DDSmerge['dayofyear']= np.atleast_1d(adjust_number_forward(DDS.isel(time=ldld)['dayofyear'].values[0]+ldld))
        
        else: 
            DDStemp = DDS.isel(time=ldld).drop('time')
            DDStemp['dayofyear']=np.atleast_1d(adjust_number_forward(DDStemp['dayofyear'].values[0]+ldld))
            DDSmerge = xr.concat([DDSmerge,DDStemp],dim='dayofyear') #we are hoping this concatenates and respects the dayofyear index
    #sort by the day of year... 
    DDSmerge_sort = DDSmerge.sortby('dayofyear')   
    #roll the result 
    DDSmerge_roll=DDSmerge_sort.rolling(dayofyear=20,center=True,min_periods=1).mean()
    
    DDSmerge_roll.to_netcdf(outdir+'MODELVARS_lead'+f"{ldld:0{3}d}"+'_climo.nc')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.6f} seconds after the load")
    print(outfile)
FNS = sorted(glob.glob(outdir+'MODELVARS_lead'+'*_climo.nc'))
DS_climo = xr.open_mfdataset(FNS,combine='nested',concat_dim='lead').assign_coords(coords={'lead':np.arange(0,46)})
DS_climo.to_netcdf(outdir+'/MODEL_VARS_full_climo.nc')